In [2]:
import os
import sys
import glob
import traceback
from datetime import datetime
import pandas as pd
from plotly.graph_objs import Scatter,Layout
import plotly
import plotly.offline as py
import numpy as np
import plotly.graph_objs as go
from plotly.subplots import make_subplots
plotly.offline.init_notebook_mode(connected=True)
import plotly.express as px
import gezi
from gezi import tqdm 
from absl import flags
FLAGS = flags.FLAGS
flags.DEFINE_string('f', '', '')
FLAGS(sys.argv)
from projects.feed.rank.src.evaluate import evaluate_df as eval
import pymp
from multiprocessing import Pool, Manager, cpu_count 
import qgrid

In [12]:
mark = 'video'
root = f'/search/odin/publicData/CloudS/chenghuige/rank/data/{mark}_hour_sgsapp_v1/exps/'
root = f'/home/gezi/tmp/rank/exps/{mark}/v25'
# root = f'/home/gezi/tmp/rank/data/{mark}_hour_sgsapp_v1/exps/v20'
# root = f'/home/gezi/tmp/rank/data/{mark}_hour_sgsapp_v1/exps/v21'
day_ = 20191224
days = 100
day = gezi.DateTime(str(day_), day=True)

In [13]:
models = [os.path.basename(x) for x in glob.glob(f'{root}/*')]
names = [
  'dlrm2',
  'baseline',
]
models

['8', 'dlrm-active', 'base', 'online']

In [14]:
dfs = Manager().list()
files = []
model_names = []
for model in models:
  files_ = glob.glob(f'{root}/{model}/infos_day/*/metrics_impression.csv')
  if not files_:
    files_ =  glob.glob(f'{root}/{model}/infos_day/*/metrics.csv')
  files += files_
  model_names += [model] * len(files_)
ps = min(len(files), cpu_count())
with pymp.Parallel(ps) as p:
  for i in tqdm(p.range(len(files))):
    df = pd.read_csv(files[i])
    df['model'] = model_names[i]
    dfs += [df]
    
dfs = list(dfs)
print(len(dfs))

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s].36it/s]

100%|██████████| 2/2 [00:00<00:00,  5.88it/s]

100%|██████████| 2/2 [00:00<00:00,  5.64it/s]

100%|██████████| 2/2 [00:00<00:00,  5.98it/s]

100%|██████████| 1/1 [00:00<00:00,  4.23it/s]

100%|██████████| 2/2 [00:00<00:00,  5.33it/s]

100%|██████████| 2/2 [00:00<00:00,  4.75it/s]


100%|██████████| 2/2 [00:00<00:00,  6.04it/s]

 50%|█████     | 1/2 [00:00<00:00,  3.94it/s]


100%|██████████| 2/2 [00:00<00:00,  5.53it/s]


100%|██████████| 2/2 [00:00<00:00,  8.15it/s]



71


In [15]:
df = pd.concat(dfs)

In [16]:
df[df.model=='base'].sort_values('date')[['date', 'group/ndcg_click']].tail()

,date,group/ndcg_click


In [17]:
# df = df[df.model.isin(['deeponly-udh-simple', 'deeponly-udh-simple-leaky'])]
# df = df[df.model.isin([x for x in models if 'torch' in x])]
# df = df[~df.model.isin(['online','16.new2'])]
# df = df[~df.model.isin(['online', 'dlrm-refresh'])]

In [18]:
df['sqrt/auc'] = (df['auc'] * df['click/time_auc']) ** 0.5
df['group/sqrt/auc'] =  (df['group/auc'] * df['group/click/time_auc']) ** 0.5
# df['quality/sqrt/auc'] = (df['quality/auc'] * df['quality/click/time_auc']) ** 0.5
# df['quality/group/sqrt/auc'] =  (df['quality/group/auc'] * df['quality/group/click/time_auc']) ** 0.5
df['rmse'] = 1. - df['mse'] ** 0.5
df['click/rmse'] = 1. - df['click/mse'] ** 0.5

In [19]:
cols = ['date', 'model','sqrt/auc', 'group/sqrt/auc', 'inv_rate']
if mark == 'video':
  cols += ['finish_auc', 'vtime_auc']
qgrid.show_grid(df[cols].sort_values(['date', 'group/sqrt/auc'], ascending=[False, False]))

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [20]:
%%html
<style>
.output_wrapper, .output {
    height:auto !important;
    max-height:500000px;  /* your desired max-height here */
}
.output_scroll {
    box-shadow:none !important;
    webkit-box-shadow:none !important;
}
</style>

In [21]:
# df[['date', 'model', 'auc', 'group/auc', 'cold/auc', 'clickmids/auc',
#     'click/time_auc','group/click/time_auc',
#    ]].sort_values(['date', 'group/auc'], ascending=[False, False]).head(10)

In [22]:
# df[[
#     'date', 'model','quality/auc', 'quality/group/auc', 
#     'quality/click/time_auc', 'quality/group/click/time_auc'
# ]].sort_values(['date', 'quality/click/time_auc'], ascending=[False, False]).head(10)

In [23]:
# df[[
#       'date', 'model', 'group/click/top1_dur_rate', 'group/click/top3_dur_rate', 'group/top1_click_rate', 'group/top3_click_rate'
# ]].sort_values(['date', 'group/top3_click_rate'], ascending=[False, False]).head(10)

### online 是线上tfrecord落盘数据直接计算 base是指定的模型比如16尾号的验证 有可能并不是当时线上全量作用于abtest 4,5,6的模型 
### 这里的验证都是针对abtest 4,5,6 小写click表示只针对正样本 Click/表示点击塔输出结果的指标 Dur/表示时长塔

In [24]:
df[['model', 'hours', 'date']].head()

,model,hours,date
0,8,22,20200415
0,8,12,20200417
0,8,18,20200416
0,8,24,20200421
0,8,24,20200419


In [25]:
df = df[df.hours >= 20]

In [26]:
# df = df[df.model.isin(['baseline', 'dlrm2'])]

In [27]:
metrics = ['hours', 
           'stats/click_ratio',
            'group/auc',
            'group/click/time_auc',
            'Click/group/auc',
            'group/auc2',
            'group/top1_click', 
            'group/ndcg3_click',
            'group/top1_dur',
           'group/sqrt/auc', 'sqrt/auc', 
           'cold/group/auc', 
#            'quality/group/auc', 'quality/group/click/time_auc', 
           'rmse', 'click/rmse', 
           'rcr_auc', 'vtime_auc', 'ptime_auc',   
            
#            'Click/group/auc',  'Click/group/auc2', 'Dur/group/click/time_auc',
          ]
gezi.line(df, x='date', y=metrics, color='model')

In [28]:
metrics = [
#            'group/sqrt/auc', 'sqrt/auc', 
#            'group/click/time_auc',
#            'quality/group/click/time_auc', 'quality/group/auc',
           'group/top1_click_rate', 'group/top3_click_rate',
           'group/click/top1_dur_rate', 'group/click/top3_dur_rate',
          ]
gezi.line(df, x='date', y=metrics, color='model')

In [29]:
metrics = [
  'group/first_click_position',
  'group/last_click_position',
  'group/ndcg_click',
  'group/ndcg3_click',
  'group/ndcg3_dur',
  'group/ndcg_dur',
#   'group/ndcg7_click',
#   'group/ndcg14_click',
  'group/click/ndcg_dur',
  'group/click/ndcg3_dur',
#   'group/click/ndcg7_dur',
#   'group/click/ndcg14_dur',
#   'quality/group/ndcg_dur',
#   'quality/group/click/ndcg_dur',
#   'quality/group/click/ndcg3_dur',
]
gezi.line(df, x='date', y=metrics, color='model')